## 1. Importación de módulos y librerías
Se importan los módulos y librerías necesarios para ejecutar el código.

In [ ]:
import requests
import os
import re
import pandas as pd
from pyproj import Transformer
import mgrs
import warnings

## 2. Descarga de datos de ubicación
Proporciona información sobre la ubicación de los taxones.
| **Columna**               | **Tipo de Dato** | **Descripción**                                                                                          |
|---------------------------|------------------|----------------------------------------------------------------------------------------------------------|
| `idtaxon`                 | Entero          | Identificador único del taxón en la base de datos.                                                      |
| `idnombre_acept`          | Decimal         | Identificador único del nombre aceptado del taxón.                                                      |
| `nombre_aceptado`         | Texto           | Nombre científico aceptado del taxón, incluyendo el autor y año de descripción.                         |
| `iddataset`               | Entero          | Identificador único del dataset relacionado.                                                            |
| `dataset`                 | Texto           | Nombre del dataset al que pertenece el registro.                                                        |
| `countrycode`             | Texto           | Código del país (ISO 3166-1 alfa-2) donde se encuentra el taxón.                                        |
| `idsistcoord`             | Entero          | Identificador único del sistema de coordenadas utilizado.                                               |
| `verbatimcoordinatesystem`| Texto           | Sistema de coordenadas utilizado en su forma literal.                                                   |
| `cuadr`                   | Texto           | Código de la cuadrícula donde se encuentra el taxón.                                                    |
| `tamaniocuadr`            | Texto           | Tamaño de la cuadrícula en kilómetros (e.g., "10").                                                     |
| `idestadocuadr`           | Entero          | Identificador único del estado de la cuadrícula.                                                        |
| `estadocuad`              | Texto           | Estado de la cuadrícula (e.g., "Confirmada").                                                           |
| `fecha`                   | Fecha           | Fecha asociada a la observación o registro del taxón.                                                   |
| `y_latitud`               | Decimal         | Latitud en coordenadas geográficas del registro del taxón.                                              |
| `x_longitud`              | Decimal         | Longitud en coordenadas geográficas del registro del taxón.                                             |
| `id`                      | Entero          | Identificador único de la ubicación en la base de datos.                                                |
| `pos`                     | Entero          | Posición del registro en la base de datos (puede usarse para orden o agrupación).                       |
| `xcentroide`              | Decimal         | Coordenada X del centroide asociado a la cuadrícula.                                                    |
| `ycentroide`              | Decimal         | Coordenada Y del centroide asociado a la cuadrícula.                                                    |
| `x`                       | Decimal         | Coordenada X de la ubicación específica del registro.                                                   |
| `y`                       | Decimal         | Coordenada Y de la ubicación específica del registro.                                                   |
| `utmcode`                 | Texto           | Código UTM asociado a la cuadrícula del registro.                                                       |
| `cuadricula`              | Texto           | Código literal de la cuadrícula asociada al registro.                                                   |
| `marino`                  | Booleano        | Indica si el registro corresponde a un área marina (`1` = sí, `0` = no).                                |
| `cod100x100`              | Texto           | Código de la cuadrícula a escala 100x100 km (si aplica).                                                |
| `perim_km`                | Decimal         | Perímetro del área registrada en kilómetros.                                                            |
| `area_km2`                | Decimal         | Área registrada en kilómetros cuadrados.                                                                |
| `cod_inb`                 | Texto           | Código interno de biodiversidad asociado al registro (si aplica).                                       |
| `orig_fid`                | Entero          | Identificador único del registro original en la base de datos.                                          |
| `area_new`                | Decimal         | Área nueva registrada (si aplica).                                                                      |
| `dif`                     | Decimal         | Diferencia en área registrada entre datos actuales y anteriores (si aplica).                            |
| `origen_dato`             | Texto           | Fuente u origen de los datos del registro (e.g., nombre del proyecto o estudio).                        |
| `geom`                    | Texto           | Información geométrica asociada al registro (e.g., geometría en formato WKT).                           |


In [ ]:
warnings.filterwarnings('ignore')
LIMIT = 30000
save_at = 100000
save_df = None
i = 16435
failed_counter = 0
FAILED_TOLERANCE = 50000
os.makedirs('Datos EIDOS', exist_ok=True)
while failed_counter < FAILED_TOLERANCE:
    if (i+1) % 50 == 0:
        print(f"Downloading {i+1}...", end='\r')
    url = f"https://iepnb.gob.es:443/api/especie/v_ubicacion?idtaxon=eq.{i}&limit={LIMIT}"
    name = f'ubicacion_{i}'
    # Realizar la petición GET
    response = requests.get(url)
    # Verificar si la solicitud fue exitosa (código 200)
    if response.status_code == 200:
        # Procesar los datos
        df = pd.DataFrame(response.json())
        if len(df) > 0:
            if len(df) == LIMIT:
                print(f'IDTaxon {i} has more than {LIMIT} records')
            if save_df is None:
                save_df = df
            else:
                save_df = pd.concat([save_df, df])
            if len(save_df) >= save_at:
                save_df.to_excel(f'Datos EIDOS/{i}.xlsx', index=False)
                save_df = None
            failed_counter = 0
        else:
            failed_counter += 1
    else:
        print(f"Error with IDTaxon {i}: {response.status_code}: {response.content}")
        failed_counter += 1
    i += 1
if save_df is not None:
    save_df.to_excel(f'Datos EIDOS/{i}.xlsx', index=False)
print(f"Stopped downloading at {i-FAILED_TOLERANCE}...")
warnings.filterwarnings('default')

Stopped downloading at 80963...


## 3. Procesado de datos

In [ ]:
# Funcion para convertir coordenadas de formato "10kmE<easting>N<northing>" a MGRS
def convert_10km_to_mgrs(coord_str: str) -> str:
    # Extraer indices de easting y northing
    match = re.match(r"10kmE(\d+)N(\d+)", coord_str)
    if not match:
        raise ValueError("El formato debe ser '10kmE<easting>N<northing>'")
    easting_idx = int(match.group(1))
    northing_idx = int(match.group(2))

    # Convertir índices a metros
    easting = easting_idx * 10000
    northing = northing_idx * 10000

    # Transformar de ETRS89/LAEA (EPSG:3035) a geográficas (EPSG:4326)
    transformer = Transformer.from_crs("EPSG:3035", "EPSG:4326", always_xy=True)
    lon, lat = transformer.transform(easting, northing)

    # Convertir a MGRS con precisión de 1 km
    mgrs_converter = mgrs.MGRS()
    mgrs_code = mgrs_converter.toMGRS(lat, lon, MGRSPrecision=1)
    return mgrs_code


# Función para ajustar valores del tipo "30TVL33" o "30TVL3231"
def ajustar_valor(cadena: str) -> str:
    # Procesa solo si la cadena empieza por alguno de los prefijos deseados
    if not (cadena.startswith("30") or cadena.startswith("29") or cadena.startswith("28") or cadena.startswith("31")):
        return cadena
    
    # Patron 1: formato "<prefijo><letras><2 dígitos>" -> se deja igual.
    patron1 = r"^((?:30|29|28|31)[A-Z]+)(\d{2})$"
    # Patron 2: formato "<prefijo><letras><2 dígitos><2 dígitos>" -> se eliminan los dos ultimos digitos.
    patron2 = r"^((?:30|29|28|31)[A-Z]+)(\d{2})(\d{2})$"
    
    if re.match(patron1, cadena):
        return cadena
    elif re.match(patron2, cadena):
        m = re.match(patron2, cadena)
        return m.group(1) + m.group(2)
    else:
        return cadena

In [ ]:
i = 0

# Procesar cada archivo Excel en la carpeta
for archivo in os.listdir('./Datos EIDOS'):
    if archivo.ends_with('.xlsx'):
        i += 1
        df = pd.read_excel(f"Datos EIDOS/{archivo}")
        
        for idx, row in df.iterrows():
            valor = row["cuadr"]
            # Si el valor es de tipo "10km...", aplicar la conversion a MGRS
            if valor.startswith("10km"):
                try:
                    nuevo_valor = convert_10km_to_mgrs(valor)
                    df.loc[idx, "cuadr"] = nuevo_valor
                except Exception as e:
                    print(f"Error en archivo {archivo}, fila {idx} con valor '{valor}': {e}")
            df.loc[idx, "cuadr"] = ajustar_valor(valor)

        df = df.drop_duplicates(subset=['idtaxon', 'cuadr'], keep="first")
        df = df.sort_values(by="idtaxon")
        df.to_excel(f"Datos EIDOS/mismo_cud_sin_repetidos_{i}.xlsx", index=False)
        os.remove(f"Datos EIDOS/{archivo}")

In [ ]:
dfs = []
total_length = 0
LENGTH_LIMIT = 100000
i = 1

# Combinar los archivos excel, ahora sin repetidos, en archivos mas grandes
for file in os.listdir('./Datos EIDOS'):
    if file.endswith('.xlsx') and file.startswith("mismo_cud_sin_repetidos_"):
        print(f"Leyendo {file}...", end='\r')
        dfs.append(pd.read_excel(f"Datos EIDOS/{file}"))
        total_length += len(dfs[-1])
        if total_length >= LENGTH_LIMIT:
            df = pd.concat(dfs)
            df.sort_values(by="idtaxon")
            print(f"Exportando archivo combinado {i}...{' '*10}", end='\r')
            df.to_excel(f"Datos EIDOS/Datos EIDOS_final_{i}.xlsx", index=False)
            i += 1
            dfs = []
            total_length = 0
if total_length > 0:
    df = pd.concat(dfs)
    df.sort_values(by="idtaxon")
    print(f"Exportando archivo combinado {i}...{' '*10}", end='\r')
    df.to_excel(f"Datos EIDOS/Datos EIDOS_final_{i}.xlsx", index=False)

# Eliminar archivos anteriores
print(f"Eliminando archivos residuales...{' '*10}", end='\r')
for file in os.listdir('./Datos EIDOS'):
    if file.endswith('.xlsx') and not file.startswith("Datos EIDOS_final_"):
        os.remove(f"Datos EIDOS/{file}")